In [1]:
import sys
sys.path.insert(1, 'Classes/')
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from multiprocessing import current_process

from ScoreGetter import ScoreGetter
from dataset_utils import checkIfEarlyMidEnd
from dataset_utils import encodeBoard
from dataset_utils import getColumns

In [2]:
df = pd.read_csv('Datasets/raw_dataset_13M.csv')
boards = df['board'].values

#### We load an engine to get a score from the positions.

In [3]:
score_getter = ScoreGetter('/home/gaetan/Téléchargements/stockfish/stockfish', 'eval', 'go depth 1')

#### We create our dataset by getting an equal number of start, middle and end game positions and encoding them.

In [4]:
total_size = 1000000
batch_size = total_size / 3.0

earlies = []
mids = []
ends = []


current = current_process()
pos = current._identity[0]-1 if len(current._identity) > 0 else 0
pbar = tqdm(total=total_size*10, desc='Splitting and encoding', position=pos)


for i in range(10):
    print(i+1)
    data = []
    for j in range(i * 1000000, i * 1000000 + total_size):
        board = boards[j]
        data.append(np.append(encodeBoard(board), score_getter.getScore(board)))
        pbar.update(1)
    df = pd.DataFrame(data, columns=np.append(getColumns(), 'cp (Stockfish 13)'))
    df.to_csv('Datasets/dataset' + str(i+1) + '.csv', index=False)
    
pbar.close()



'''
for i in range(4000000, len(boards)):
    board = boards[i]
    part = checkIfEarlyMidEnd(board)
    
    if len(earlies) < batch_size and part == "early_game":
        earlies.append(np.append(encodeBoard(board), score_getter.getScore(board)))
        pbar.update(1)
    
    elif len(mids) < batch_size and part == "mid_game":
        mids.append(np.append(encodeBoard(board), score_getter.getScore(board)))
        pbar.update(1)
        
    elif len(ends) < batch_size and part == "end_game":
        ends.append(np.append(encodeBoard(board), score_getter.getScore(board)))
        pbar.update(1)
    
    if len(earlies) >= batch_size and len(mids) >= batch_size and len(ends) >= batch_size:
        break
    
pbar.close()
'''

Splitting and encoding:   0%|          | 1/10000000 [00:00<477:23:12,  5.82it/s]

1


Splitting and encoding:  10%|█         | 1000099/10000000 [20:15<571:43:07,  4.37it/s]

2


Splitting and encoding:  20%|██        | 2000090/10000000 [40:37<620:07:20,  3.58it/s]

...

9


Splitting and encoding:  90%|█████████ | 9000060/10000000 [3:07:13<81:28:06,  3.41it/s] 

10


Splitting and encoding: 100%|██████████| 10000000/10000000 [3:27:53<00:00, 801.69it/s] 


'\nfor i in range(4000000, len(boards)):\n    board = boards[i]\n    part = checkIfEarlyMidEnd(board)\n    \n    if len(earlies) < batch_size and part == "early_game":\n        earlies.append(np.append(encodeBoard(board), score_getter.getScore(board)))\n        pbar.update(1)\n    \n    elif len(mids) < batch_size and part == "mid_game":\n        mids.append(np.append(encodeBoard(board), score_getter.getScore(board)))\n        pbar.update(1)\n        \n    elif len(ends) < batch_size and part == "end_game":\n        ends.append(np.append(encodeBoard(board), score_getter.getScore(board)))\n        pbar.update(1)\n    \n    if len(earlies) >= batch_size and len(mids) >= batch_size and len(ends) >= batch_size:\n        break\n    \npbar.close()\n'

In [5]:
'''
data = earlies + mids + ends
random.shuffle(data)
len(data)
'''

'\ndata = earlies + mids + ends\nrandom.shuffle(data)\nlen(data)\n'

In [6]:
'''
df = pd.DataFrame(data, columns=np.append(getColumns(), 'cp (Stockfish 13)'))
df.to_csv('Datasets/dataset5.csv', index=False)
df.head()
'''

"\ndf = pd.DataFrame(data, columns=np.append(getColumns(), 'cp (Stockfish 13)'))\ndf.to_csv('Datasets/dataset5.csv', index=False)\ndf.head()\n"

#### We create a test dataset.

In [7]:
'''
test_size = 1000000
tests = []

current = current_process()
pos = current._identity[0]-1 if len(current._identity) > 0 else 0
pbar = tqdm(total=test_size, desc='Encoding', position=pos)

tot_size = boards.shape[0]
for i in range(test_size):
    idx = np.random.randint(tot_size)
    tests.append(np.append(encodeBoard(boards[idx]), score_getter.getScore(boards[idx])))
    pbar.update(1)
pbar.close()

len(tests)
'''

"\ntest_size = 1000000\ntests = []\n\ncurrent = current_process()\npos = current._identity[0]-1 if len(current._identity) > 0 else 0\npbar = tqdm(total=test_size, desc='Encoding', position=pos)\n\ntot_size = boards.shape[0]\nfor i in range(test_size):\n    idx = np.random.randint(tot_size)\n    tests.append(np.append(encodeBoard(boards[idx]), score_getter.getScore(boards[idx])))\n    pbar.update(1)\npbar.close()\n\nlen(tests)\n"

In [8]:
'''
df = pd.DataFrame(tests, columns = np.append(getColumns(), 'cp (Stockfish 13)'))
df.to_csv('Datasets/test_dataset.csv', index=False)
df.head()
'''

"\ndf = pd.DataFrame(tests, columns = np.append(getColumns(), 'cp (Stockfish 13)'))\ndf.to_csv('Datasets/test_dataset.csv', index=False)\ndf.head()\n"